<a href="https://colab.research.google.com/github/KolatimiDave/Hamoye/blob/master/StageC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')

Importing Relevant Packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV

In [3]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv")

In [4]:
df.head(3)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable


In [28]:
df.shape

(10000, 13)

In [27]:
sum(df.isna().sum())

0

In [5]:
df.drop('stab',1,inplace=True)

In [6]:
def convert_stabf(x):
  if x == 'stable':
    return 1
  elif x == 'unstable':
    return 0
  else:
    pass

In [7]:
df['stabf'] = df['stabf'].apply(convert_stabf)


In [9]:
X = df.drop('stabf',1)
y = df['stabf']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [11]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
lgbm = LGBMClassifier(random_state = 1)
rfc = RandomForestClassifier(random_state = 1)
xgb = XGBClassifier(random_state = 1)

In [15]:
def model(x, X_test, y_test, X_train = X_train, y_train = y_train):
  x.fit(X_train, y_train)
  score = x.score(X_test, y_test)
  return score  

In [17]:
lgbm_score = model(lgbm,X_test=X_test, y_test=y_test)
print(lgbm_score)

0.9375


In [18]:
rfc_score = model(rfc,X_test=X_test, y_test=y_test)
print(rfc_score)

0.9295


In [19]:
xgb_score = model(xgb,X_test=X_test, y_test=y_test)
print(xgb_score)

0.9195


In [22]:
extc = ExtraTreesClassifier(random_state = 1)

In [23]:
params = {
    'n_estimators' : [50, 100, 300, 500, 1000],
    'min_samples_split' : [2, 3, 5, 7, 9],
    'min_samples_leaf' :[1, 2, 4, 6, 8],
    'max_features' : ['auto', 'sqrt', 'log2', None]
}

In [24]:
rscv = RandomizedSearchCV(estimator=extc,param_distributions=params,cv=5,n_iter=10,scoring='accuracy',n_jobs=-1,verbose=1, random_state=1)

In [25]:
rscv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                        

In [46]:
rscv.best_params_

{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [29]:
rscv.best_estimator_.score(X_test,y_test)

0.927

In [30]:
extc_1 = ExtraTreesClassifier(random_state=1)

In [31]:
extc_1.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                     n_jobs=None, oob_score=False, random_state=1, verbose=0,
                     warm_start=False)

In [32]:
extc_1.score(X_test,y_test)

0.9285

In [34]:
importances = pd.Series(extc_1.feature_importances_, index=X.columns.to_list())

In [36]:
importances.sort_values()

p1      0.039507
p2      0.040371
p4      0.040579
p3      0.040706
g1      0.089783
g2      0.093676
g4      0.094019
g3      0.096883
tau3    0.113169
tau4    0.115466
tau1    0.117397
tau2    0.118445
dtype: float64